In [33]:
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import os
import skimage as ski 
from skimage import io
io.use_plugin('pil')
import PIL
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

In [17]:
def normalize_images(images):
    min_value = np.min(images)
    max_value = np.max(images)
    normalized_images = (images - min_value) / (max_value - min_value)
    return normalized_images

In [40]:
path_name = "//home//rounak//Desktop//KN//"
image_files = [file for file in os.listdir(path_name) if file.endswith(".tiff")]
dataset = []
k=0
for image_file in image_files:
    image_path = os.path.join(path_name,image_file)
    img = io.imread(image_path, plugin='pil')
    dataset.append(normalize_images(np.expand_dims(img[:,:,0], axis=0)))

In [48]:
dataset = np.float32(np.array(dataset))

In [55]:
dataset_1 = []
for i in range(5067):
    dataset_1.append(dataset[i][:, :320, :320])
dataset_1 = np.array(dataset_1)

In [56]:
class DownBlock(nn.Module):
    def __init__(self, in_channels, out_channels, device="cuda:1"):
        super(DownBlock, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.model = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)


class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels, flag = 1, device="cuda:1"):
        super(UpBlock, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        if(flag == 1):
            self.model = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, 1, 1),
                nn.ReLU(),
                nn.ConvTranspose2d(out_channels, out_channels, 2, 2),
            )
        else:
            self.model = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, 1, 1),
                nn.ReLU()
            )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

    
class AutoEncoder(nn.Module):
    def __init__(self, in_channels_):
        super(AutoEncoder, self).__init__()
        self.in_channels_ = in_channels_
        layers_1 = [
            DownBlock(in_channels=in_channels_, out_channels=16),
            DownBlock(16, 8),
            DownBlock(8, 8)
        ]
        layers_2 = [
            UpBlock(in_channels = 8, out_channels = 8),
            UpBlock(8, 8),
            UpBlock(8, 16),
            UpBlock(16, in_channels_, 2)
        ]
        layers = [*layers_1, *layers_2]
        self.model = nn.Sequential(*layers)
        self.model_decode = nn.Sequential(*layers_1)
        self.model_encode = nn.Sequential(*layers_2)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

    
    
auto_encoder = AutoEncoder(1).to("cpu")
dataloader = DataLoader(dataset_1, batch_size=256, shuffle=True)
optim = torch.optim.Adam(auto_encoder.parameters(), lr=1e-4)


In [57]:
import os
auto_encoder.train()
loss_lst=[]
loss_val = 0
for ep in range(0, 100):
    print("epoch number: ", ep)
    optim.param_groups[0]['lr'] = 1e-3
    pbar = tqdm(dataloader, mininterval = 2)
    loss_val = 0
    for x in pbar:
        optim.zero_grad()
        x=x.to("cpu")
        pred_x = auto_encoder(x)
        loss = F.binary_cross_entropy(pred_x, x)
        loss_val += loss
        loss.backward()
        optim.step()
    loss_lst.append(loss_val)



epoch number:  0


  0%|          | 0/20 [00:39<?, ?it/s]


RuntimeError: [enforce fail at C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1677721600 bytes.